<a href="https://colab.research.google.com/github/abhisheksuran/Reinforcement_Learning/blob/master/Reinforce(PG)_reuploaded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
import gym 

In [ ]:
!pip3 install box2d-py

     |████████████████████████████████| 450kB 2.8MB/s 


In [ ]:
env= gym.make("LunarLander-v2")
low = env.observation_space.low
high = env.observation_space.high

In [ ]:
class model(tf.keras.Model):
  def __init__(self):
    super().__init__()
    self.d1 = tf.keras.layers.Dense(64,activation='relu')
    self.d2 = tf.keras.layers.Dense(64,activation='relu')
    self.out = tf.keras.layers.Dense(4,activation='softmax')

  def call(self, input_data):
    x = tf.convert_to_tensor(input_data)
    x = self.d1(x)
    x = self.d2(x)
    x = self.out(x)
    return x

  def action_value(self, state):
        p = self.predict(state)
        return p


In [ ]:
class agent():
  def __init__(self):
    self.net = model()
    opt = tf.keras.optimizers.Adam(learning_rate=0.001)
    self.net.compile(optimizer=opt, loss='categorical_crossentropy')
    self.gamma = 1

  def act(self,state):
    action = np.random.choice([i for i in range(env.action_space.n)], 1, p=self.net.action_value(np.array([state]))[0])
    
    return action[0]

  def train(self, states, rewards):
    sum_reward = 0
    discnt_rewards = []
    rewards.reverse()
    for r in rewards:
      sum_reward = r + self.gamma*sum_reward
      discnt_rewards.append(sum_reward)
    discnt_rewards.reverse()  
    discnt_rewards = np.array(discnt_rewards)
    discnt_rewards -= np.mean(discnt_rewards)
    discnt_rewards /= np.std(discnt_rewards)
    #states  = np.array(states)
    states = np.vstack(states)
    print(states.shape)
    self.net.train_on_batch(states, discnt_rewards)
    



In [ ]:
agentoo7 = agent()
steps = 10000
for s in range(steps):
  
  done = False
  state = env.reset()
  total_reward = 0
  rewards = []
  states = []
  while not done:
    #env.render()
    action = agentoo7.act(state)
    #print(action)
    next_state, reward, done, _ = env.step(action)
    rewards.append(reward)
    states.append(state)
    state = next_state
    total_reward += reward
    
    if done:
      agentoo7.train(states, rewards)
      #print("total step for this episord are {}".format(t))
      print("total reward after {} steps is {}".format(s, total_reward))